In [35]:
!cd /home/enis/projects/nna/src/nna/exp/megan/run-5

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchaudio
torchaudio.set_audio_backend('sox_io')

import numpy as np
import pandas as pd

import os

/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


/home/enis/projects/inferences/megantaxo/happy-disco-3229/megantaxo


In [33]:
from src.megantaxo import metrics

In [2]:

from nna import dataimport
from nna import fileUtils
from nna.exp import runutils

import modelarchs  # type: ignore


In [3]:


def repeat_data(data, expected_len) -> np.ndarray:
    '''pad by zeros if it is not divisible expected_len seconds.
    '''
    sr = 48000
    left_over = (data.shape[0]) % (expected_len * sr)

    if left_over != 0:
        missing_element_count = (expected_len * sr) - left_over
        padded_data = np.pad(data[-left_over:], (0, missing_element_count),
                             'constant',
                             constant_values=(0, 0))
        return np.concatenate([data[:-left_over], padded_data])  # type: ignore
    else:
        return data



In [28]:
class pathMap():
    def __init__(self,model_relative_path=None) -> None:
        scratch = '/scratch/enis/data/nna/'
        home = '/home/enis/projects/nna/'
        self.run_id = 'run-5'
        self.exp_dir = f'/home/enis/projects/nna/src/nna/exp/megan/{self.run_id}/'
        self.clipping_results_path = (scratch +
                                         'clipping_info/all-merged_2021-02-10/')
        self.output_dir = scratch + 'real/'
        self.file_properties_df_path = scratch + 'database/allFields_dataV4.pkl'
        # model_path= ('/home/enis/projects/nna/src/nna/exp/megan/run-3/'+
        # 'checkpoints_keep/glorious-sweep-57/best_model_56_ROC_AUC=0.8690.pt')
        checkpoints_dir = scratch + f'runs_models/megan/{self.run_id}/checkpoints/'
        if model_relative_path is None:
            self.model_relative_path = 'comic-sweep-29/best_model_20_min_ROC_AUC=0.7202.pt'
        else:
            self.model_relative_path =model_relative_path
#         self.model_relative_path = 'warm-sweep-160'+'/'+'best_model_11_min_ROC_AUC=0.7076.pt'
    
        self.model_path_name = self.model_relative_path.replace('/','_')
        self.model_path = (
            checkpoints_dir +
            self.model_relative_path)
        self.out_csv_file_name = scratch+ f'labeling/megan/AudioSamplesPerSite_post/{self.run_id}_{self.model_path_name}_labels_v0.csv'



def setup_inputs(args):
    index, count = int(args.index), int(args.count)

    region_location = [['dalton', '01'], ['dalton', '02'], ['dalton', '03'],
                       ['dalton', '04'], ['dalton', '05'], ['dalton', '06'],
                       ['dalton', '07'], ['dalton', '08'], ['dalton', '09'],
                       ['dalton', '10'], ['dempster', '11'], ['dempster', '12'],
                       ['dempster', '13'], ['dempster',
                                            '14'], ['dempster', '16'],
                       ['dempster', '17'], ['dempster',
                                            '19'], ['dempster', '20'],
                       ['dempster', '21'], ['dempster',
                                            '22'], ['dempster', '23'],
                       ['dempster', '24'], ['dempster', '25'],
                       ['ivvavik', 'AR01'], ['ivvavik', 'AR02'],
                       ['ivvavik', 'AR03'], ['ivvavik', 'AR04'],
                       ['ivvavik', 'AR05'], ['ivvavik', 'AR06'],
                       ['ivvavik', 'AR07'], ['ivvavik', 'AR08'],
                       ['ivvavik', 'AR09'], ['ivvavik', 'AR10'],
                       ['ivvavik', 'SINP01'], ['ivvavik', 'SINP02'],
                       ['ivvavik', 'SINP03'], ['ivvavik', 'SINP04'],
                       ['ivvavik', 'SINP05'], ['ivvavik', 'SINP06'],
                       ['ivvavik', 'SINP07'], ['ivvavik', 'SINP08'],
                       ['ivvavik', 'SINP09'], ['ivvavik', 'SINP10']]

    return region_location[index:index + count]


def setup(args,config=None):

    pathmap = pathMap(model_relative_path=args.model_name)

    os.chdir(pathmap.exp_dir)

    file_properties_df = pd.read_pickle(pathmap.file_properties_df_path)

    device = 'cuda:' + str(args.gpu)
    device = torch.device(device)

    CATEGORY_COUNT = 9
    # '1.1.10','1.1.7'
    maxMelLen = 938
    ToTensor_ins = modelarchs.ToTensor(maxMelLen, 48000, device)
    transformCompose = transforms.Compose([
        ToTensor_ins,
    ])

    h_w = [128, 938]
    
    if config is None:
        config = {}
    config['label_names'] = ['1-0-0',
                            '1-1-0',
                            '1-1-10',
                            '1-1-7',
                            '0-0-0',
                            '1-3-0',
                            '1-1-8',
                            '0-2-0',
                            '3-0-0',
                            ]
    #warm-sweep-160/
    config['v_str'] = 'multi9-V1'
#     config['fc_1_size'] = 64
#     config['CNN_kernel_size'] = 12
#     config['CNN_filters_1'] = 6
    config['expected_len'] = 10
    config['device'] = device
    
#     #dandy-sweep-196
#     config['v_str'] = 'multi9-V1'
#     config['fc_1_size'] = 71
#     config['CNN_kernel_size'] = 12
#     config['CNN_filters_1'] = 16
#     config['expected_len'] = 10
#     config['device'] = device
    
#comic-sweep-29
#     config['v_str'] = 'multi9-V1'
#     config['fc_1_size'] = 64
#     config['CNN_kernel_size'] = 12
#     config['CNN_filters_1'] = 6
#     config['expected_len'] = 10
#     config['device'] = device

    output_shape = (CATEGORY_COUNT,)

    model_saved = modelarchs.singleconv1dModel(
        out_channels=config['CNN_filters_1'],
        h_w=(1, h_w[0] * h_w[1]),
        fc_1_size=config['fc_1_size'],
        kernel_size=config['CNN_kernel_size'],
        output_shape=output_shape)

    model_saved.load_state_dict(
        torch.load(pathmap.model_path, map_location=config['device']))
    model_saved.eval().to(config['device'])

    return model_saved, transformCompose, config, file_properties_df, pathmap






In [5]:

def prepare_dataloader_from_array(input_data):

    # divide to 10 second excerpts
    print(input_data.shape)
    input_file_data = input_data.reshape(-1, 480000)
    input_file_data = torch.from_numpy(input_file_data).float()
    dataset = {
        'predict':
            runutils.audioDataset(input_file_data,
                                  None,
                                  transform=transformCompose)
    }
    dataloader = {
        'predict':
            torch.utils.data.DataLoader(dataset['predict'],
                                        shuffle=False,
                                        batch_size=1)
    }
    return dataloader



In [6]:
class Arguments():
    def __init__(self,index,count,gpu,model_name):
        self.index=index
        self.count=count
        self.gpu=gpu
        self.model_name=model_name



In [7]:
# main(args)

In [8]:
# region_location = setup_inputs(args)
# model_saved, transformCompose, config, file_properties_df, pathmap = setup(args)



In [9]:
# def load_audio_files(region_location, file_properties_df):
#     '''Load audio files from given regions and locations as Audio dataset
#     '''
#     region_location_datasets = {}
#     for region, location in region_location:
#         filtered_files = file_properties_df[file_properties_df.region == region]
#         filtered_files = filtered_files[filtered_files.locationId == location]
#         filtered_files = filtered_files[filtered_files.durationSec > 0]
#         dataset_name_v = '-'.join([region, location])
#         audio_dataset = dataimport.Dataset(dataset_name_v=dataset_name_v)
#         for i in filtered_files.iterrows():
#             audio_dataset[i[0]] = dataimport.Audio(i[1].name,
#                                                    float(i[1].durationSec))
#         region_location_datasets[(region, location)] = audio_dataset
#     return region_location_datasets

In [10]:
import run

In [11]:
audio_dataset, _ = run.prepare_dataset()


/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite/meganLabeledFiles_wlenV1.txt
/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite
4 files are missing corresponding to excell entries
'-> 5 number of samples are DELETED due to ignore_files and missing_audio_files'
-> 415 samples DELETED because they are not in the excell

-> 0 samples DELETED because they do not have the taxo info coming from excell

-> classes that do not have enough data:
[REMOVED!]

-> classes that have enough data:
['other-biophony'] 56.0
['other-insect'] 140.0
['other-bird'] 661.0
['songbirds'] 392.0
['duck-goose-swan'] 183.0
['grouse-ptarmigan'] 59.0
['other-anthrophony'] 66.0
['other-mammal'] 0.0
['other-silence'] 20.0
['unknown-sound'] 2.0
['other-aircraft'] 107.0
['seabirds'] 1.0
['canids'] 1.0
['loons'] 29.0
['other-car'] 37.0
['other-flare'] 11.0
['other-rain'] 20.0
('-> 0 number of samples are deleted because their taxonomy category does not '
 'have enough data.')
-> classes that do not have e

In [12]:
EXCERPT_LENGTH=10
audio_dataset = run.dataset_generate_samples(audio_dataset,EXCERPT_LENGTH)
all_taxo= list({sound_ins.taxo_code for sound_ins in audio_dataset.values()})

x_data, y_data, location_id_info = run.put_samples_into_array(
    all_taxo, [], audio_dataset)
x_data=np.array(x_data)

In [13]:
names=list(iter(audio_dataset.keys()))

In [14]:
import datetime


timestamps=[]
file_paths = []
audio_index=[]
for sound_ins in audio_dataset.values():
    i=0
    for sample in sound_ins.samples:
        
        day,clock = (sound_ins.name.split('_')[1:3])
        year,month,day = day[:4],day[4:6],day[6:8]
        hour,minute,second = clock[:2],clock[2:4],clock[4:6]
        year,month,day=int(year),int(month),int(day)
        hour,minute,second=int(hour),int(minute),int(second)
        timestamp = datetime.datetime(year,
                                  month,
                                  day,
                                  hour=hour,
                                  minute=minute,
                                  second=second,
                                  microsecond=0)
        timestamp=timestamp+datetime.timedelta(seconds=10*i)
        timestamps.append(timestamp)
        audio_index.append(i)
        file_paths.append(str(sound_ins.path))
        i+=1


In [15]:
#     dataloader = prepare_dataloader_from_audio_ins(
#         audio_ins, config, transformCompose)

In [16]:
#     outputs = single_file_inference(dataloader, config, model_saved

In [17]:
x_data=np.array(x_data)

In [19]:
# dataloader=prepare_dataloader_from_array(x_data[:])

In [20]:
# outputs = single_file_inference(dataloader, config, model_saved)

In [21]:
def sigmoid(data):
    return 1 / (1 + np.exp(-data))

In [22]:
sigmoid(np.array([1,2,3,-2]))

array([0.73105858, 0.88079708, 0.95257413, 0.11920292])

In [23]:
x_data.shape

(3083, 480000)

In [24]:
# SAME MODEL FROM HERE

In [25]:


def roc_auc_perClass_compute_fn(y_pred, y_true):
    try:
        from sklearn.metrics import roc_auc_score
    except ImportError:
        raise RuntimeError(
            "This contrib module requires sklearn to be installed.")

    res = roc_auc_score(y_true, y_pred, average=None)
    return res

In [26]:
loc_per_set = [[
    '12', '14', '27', '49', '31', '39', '44', '45', '48', '19', '16', '21',
    '38', '41', '20', '29', '37', '15'
], ['17', '46', '50', '32', '33', '25', '40'],
               ['11', '18', '34', '24', '13', '22', '36', '47', '30']]

target_taxo = [
    '1.0.0', '1.1.0', '1.1.10', '1.1.7', '0.0.0', '1.3.0', '1.1.8', '0.2.0',
    '3.0.0'
]
multi_label_vector = run.create_multi_label_vector(target_taxo, y_data)



def loc_per_set_find(loc,loc_per_set):
    i2name={0:'train',1:'test',2:'valid'}
    for i,m in enumerate(loc_per_set):
        if loc in m:
            return i2name[i]

set_names=[loc_per_set_find(loc,loc_per_set) for loc in location_id_info]


target_taxo_names=[audio_dataset.taxonomy[taxo][0].replace('other-','') for taxo in target_taxo]
target_taxo_names_header=('_pred,'.join(target_taxo_names))+'_pred'
target_taxo_header = (','.join(target_taxo_names))

In [171]:

#/scratch/enis/data/nna/runs_models/megan/run-5/checkpoints/



config={}
res_res=[]
for model_name in [ 
#     'dandy-sweep-196/best_model_68_min_ROC_AUC=0.7063.pt',
#  'dandy-sweep-196/best_model_67_min_ROC_AUC=0.7062.pt',
 
#  'deep-sweep-198/best_model_12_min_ROC_AUC=0.7000.pt',
#  'deep-sweep-198/best_model_14_min_ROC_AUC=0.6982.pt',
 
#  'fearless-sweep-206/best_model_64_min_ROC_AUC=0.6955.pt',
#  'fearless-sweep-206/best_model_66_min_ROC_AUC=0.6975.pt',
 
#  'sweepy-sweep-203/best_model_36_min_ROC_AUC=0.7025.pt',
# 'sweepy-sweep-203/best_model_37_min_ROC_AUC=0.7036.pt', 
 
#  'visionary-sweep-121/best_model_145_min_ROC_AUC=0.6967.pt',
# 'visionary-sweep-121/best_model_148_min_ROC_AUC=0.6976.pt',
 
#  'warm-sweep-160/best_model_10_min_ROC_AUC=0.7065.pt',
# 'warm-sweep-160/best_model_11_min_ROC_AUC=0.7076.pt',
                   
 'comic-sweep-29/best_model_20_min_ROC_AUC=0.7202.pt',
# 'comic-sweep-29/best_model_24_min_ROC_AUC=0.7188.pt' 
]:
    
    print(model_name)
    
    if 'dandy-sweep-196' in model_name:
        config['fc_1_size'] = 71
        config['CNN_kernel_size'] = 12
        config['CNN_filters_1'] = 16

    elif 'deep-sweep-198' in model_name:
        config['CNN_filters_1'] = 5
        config['CNN_kernel_size'] = 12
        config['fc_1_size'] = 84
    
    elif 'fearless-sweep-206' in model_name:
        config['CNN_filters_1'] = 5
        config['CNN_kernel_size'] = 12
        config['fc_1_size'] = 115
        
    elif 'sweepy-sweep-203' in model_name:
        config['CNN_filters_1'] = 12
        config['CNN_kernel_size'] = 12
        config['fc_1_size'] = 87
    
    elif 'visionary-sweep-121' in model_name:
        config['CNN_filters_1'] = 4
        config['CNN_kernel_size'] = 12
        config['fc_1_size'] = 84
    elif 'warm-sweep-160' in model_name:
        config['CNN_filters_1'] = 9
        config['CNN_kernel_size'] = 10
        config['fc_1_size'] = 110
        
    elif 'comic-sweep-29' in model_name:
        config['fc_1_size'] = 64
        config['CNN_kernel_size'] = 12
        config['CNN_filters_1'] = 6
    
    args = Arguments(0,1,1,model_name)
    model_saved, transformCompose, config, file_properties_df, pathmap = setup(args,config=config)


    dataloader=prepare_dataloader_from_array(x_data[:])

    # model = MyModel()
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    model_saved.fc1.register_forward_hook(get_activation('fc1'))
    # x = torch.randn(1, 25)
    # output = model(x)
    # print(activation['fc2'])

    # def single_file_inference(dataloader, config, model_saved):
    outputs = []
    activations=[]
    for inputs, labels in dataloader['predict']:
    #     print(labels)
        del labels
        inputs = inputs.float().to(config['device'])

        output = model_saved(inputs)
        output = output.to('cpu')
        index = output.data.numpy()
        outputs.append(index)
        activations.append(activation['fc1'])
    # outputs.append(index)
    #     outputs = np.concatenate(outputs)
    #     return outputs
    
    
    
    

    # outputs_list=[]
    # for out in outputs:
    #     out_str=[ f"{x:.2f}" for x in list(sigmoid(out[0]))]
    #     outputs_list.append(out_str)

    outputs_list=[]
    outputs_full_list=[]
    outputs_sigmoid_full_list=[]
    for out in outputs:
        out_str=[ f"{x:.2f}" for x in list(torch.sigmoid(torch.tensor(out[0])))]
        out_strfull=[ f"{x}" for x in list((out[0]))]
        out_str_sigmoid_full=[ f"{x}" for x in list(torch.sigmoid(torch.tensor(out[0])))]

        outputs_list.append(out_str)
        outputs_full_list.append(out_strfull)
        outputs_sigmoid_full_list.append(out_str_sigmoid_full)

#     activationsnp=[]
#     for act in activations:
#         act = act.to('cpu')
#         act = act.data.numpy()
#         activationsnp.append(act)

#     activationsnp=np.array(activationsnp)


    # audio_index,file_paths
    # pathmap.out_csv_file_name
    with open(pathmap.out_csv_file_name,'w') as f:
        f.writelines('sample_id,file_path,part_index,timestamp,location_id,set_name,'+target_taxo_header+','+target_taxo_names_header+'\n')
        for i,label_vector in enumerate(multi_label_vector):
            label_vector = [str(i) for i in label_vector]
            label_vector=','.join(label_vector)
            pred_vector = outputs_sigmoid_full_list[i]
            t = timestamps[i].strftime('%Y-%m-%d_%H:%M:%S')
            f.writelines(f'{i},{file_paths[i]},{audio_index[i]},{t},{location_id_info[i]},{set_names[i]},{(label_vector)},'+(','.join(pred_vector))+'\n')

    asd=pd.read_csv(pathmap.out_csv_file_name)
    preds=[i for i in asd.columns if 'pred' in i]
    actual=[i.split('_')[0] for i in preds ]
    valid=asd[asd['set_name']=='valid']
    VALID_KK=roc_auc_perClass_compute_fn(np.array(valid[preds]),np.array(valid[actual]))

    valid=asd[asd['set_name']=='test']
    TEST_KK=roc_auc_perClass_compute_fn(np.array(valid[preds]),np.array(valid[actual]))
    
    print(pathmap.out_csv_file_name)
    print(TEST_KK)
    print(VALID_KK)
    res_res.append([pathmap.out_csv_file_name,TEST_KK,VALID_KK])

comic-sweep-29/best_model_20_min_ROC_AUC=0.7202.pt
(3083, 480000)
/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_comic-sweep-29_best_model_20_min_ROC_AUC=0.7202.pt_labels_v0.csv
[0.69217629 0.68439965 0.3564696  0.68679907 0.80175217 0.62494624
 0.62062741 0.58730659 0.6942959 ]
[0.7256433  0.7512538  0.723035   0.81626475 0.80800869 0.91174769
 0.72019676 0.81818182 0.75679074]


In [169]:
len(out_str_sigmoid_full)

9

In [172]:
pathmap.out_csv_file_name

'/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_comic-sweep-29_best_model_20_min_ROC_AUC=0.7202.pt_labels_v0.csv'

In [41]:
from tabulate import tabulate
print(tabulate([['Alice', 24], ['Bob', 19]], headers=['Name', 'Age']))

Name      Age
------  -----
Alice      24
Bob        19


In [53]:
mydict={}
table_list=[]
test_list=[]
valid_list=[]
i=0
for name,test,valid in res_res:
    short_name=name.split('/')[-1]
    short_name=short_name.split('_')
    short_name = short_name[1]+'_epoch-'+short_name[4]
#     print(short_name)
    table_list.append(('valid_'+short_name,*valid))
    table_list.append(('test_'+short_name,*test))
    test_list.append((short_name,*test))
    valid_list.append((short_name,*valid))
    
    i+=2
    

In [54]:
# outputs[0][0]

In [55]:
print(tabulate(table_list))

-----------------------------------  --------  --------  --------  --------  --------  --------  --------  --------  --------
valid_dandy-sweep-196_epoch-68       0.712079  0.708273  0.706325  0.78401   0.814524  0.921007  0.744792  0.748295  0.726735
test_dandy-sweep-196_epoch-68        0.699286  0.681486  0.350011  0.751324  0.831858  0.62086   0.706834  0.881923  0.735294
valid_dandy-sweep-196_epoch-67       0.715728  0.70795   0.706158  0.772538  0.813865  0.922743  0.75      0.76461   0.71831
test_dandy-sweep-196_epoch-67        0.708333  0.685437  0.355081  0.744743  0.833574  0.657706  0.722893  0.883918  0.727273
valid_deep-sweep-198_epoch-12        0.713864  0.793598  0.729059  0.700015  0.738056  0.806134  0.707755  0.738149  0.706992
test_deep-sweep-198_epoch-12         0.607974  0.654411  0.46175   0.699961  0.752228  0.645305  0.619445  0.831058  0.772646
valid_deep-sweep-198_epoch-14        0.722249  0.791489  0.719832  0.698184  0.782936  0.836516  0.7011    0.729302  0.

In [68]:
headers = ['(test) model_name']+target_taxo_names
print(tabulate(test_list,headers=headers))

(test) model_name                biophony      bird    songbirds    duck-goose-swan    anthrophony    insect    grouse-ptarmigan    aircraft    silence
-----------------------------  ----------  --------  -----------  -----------------  -------------  --------  ------------------  ----------  ---------
dandy-sweep-196_epoch-68         0.699286  0.681486     0.350011           0.751324       0.831858  0.62086             0.706834    0.881923   0.735294
dandy-sweep-196_epoch-67         0.708333  0.685437     0.355081           0.744743       0.833574  0.657706            0.722893    0.883918   0.727273
deep-sweep-198_epoch-12          0.607974  0.654411     0.46175            0.699961       0.752228  0.645305            0.619445    0.831058   0.772646
deep-sweep-198_epoch-14          0.622251  0.666988     0.461309           0.689486       0.767371  0.667599            0.642537    0.834384   0.774834
fearless-sweep-206_epoch-64      0.712895  0.71533      0.344014           0.717523     

In [69]:
headers = ['(valid) model_name']+target_taxo_names
print(tabulate(valid_list,headers=headers))

(valid) model_name               biophony      bird    songbirds    duck-goose-swan    anthrophony    insect    grouse-ptarmigan    aircraft    silence
-----------------------------  ----------  --------  -----------  -----------------  -------------  --------  ------------------  ----------  ---------
dandy-sweep-196_epoch-68         0.712079  0.708273     0.706325           0.78401        0.814524  0.921007            0.744792    0.748295   0.726735
dandy-sweep-196_epoch-67         0.715728  0.70795      0.706158           0.772538       0.813865  0.922743            0.75        0.76461    0.71831
deep-sweep-198_epoch-12          0.713864  0.793598     0.729059           0.700015       0.738056  0.806134            0.707755    0.738149   0.706992
deep-sweep-198_epoch-14          0.722249  0.791489     0.719832           0.698184       0.782936  0.836516            0.7011      0.729302   0.711896
fearless-sweep-206_epoch-64      0.669377  0.708758     0.69552            0.784442      

In [354]:
# with open('/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_comic-sweep-29_activations-fc1_v3.npy', 'wb') as f:
#     np.save(f,activationsnp)


In [150]:
for h,i in zip(headers[1:],valid_list[-1][1:]):
    print(f'{h}:{i:.2f}')

biophony:0.75
bird:0.75
songbirds:0.72
duck-goose-swan:0.84
anthrophony:0.82
insect:0.90
grouse-ptarmigan:0.73
aircraft:0.82
silence:0.78


In [148]:
print(tabulate([['none']+[f'{i:.2f}' for i in valid_list[-1][1:]]],headers=headers))

(valid) model_name      biophony    bird    songbirds    duck-goose-swan    anthrophony    insect    grouse-ptarmigan    aircraft    silence
--------------------  ----------  ------  -----------  -----------------  -------------  --------  ------------------  ----------  ---------
none                        0.75    0.75         0.72               0.84           0.82       0.9                0.73        0.82       0.78


In [356]:
(valid) model_name               biophony      bird    songbirds    duck-goose-swan    anthrophony    insect    grouse-ptarmigan    aircraft    silence
-----------------------------  ----------  --------  -----------  -----------------  -------------  --------  ------------------  ----------  ---------
dandy-sweep-196_epoch-68         0.712079  0.708273     0.706325           0.78401        0.814524  0.921007            0.744792    0.748295   0.726735
dandy-sweep-196_epoch-67         0.715728  0.70795      0.706158           0.772538       0.813865  0.922743            0.75        0.76461    0.71831
deep-sweep-198_epoch-12          0.713864  0.793598     0.729059           0.700015       0.738056  0.806134            0.707755    0.738149   0.706992
# deep-sweep-198_epoch-14          0.722249  0.791489     0.719832           0.698184       0.782936  0.836516            0.7011      0.729302   0.711896
fearless-sweep-206_epoch-64      0.669377  0.708758     0.69552            0.784442       0.802316  0.826968            0.783565    0.807224   0.750755
# fearless-sweep-206_epoch-66      0.670419  0.707798     0.69748            0.787062       0.806627  0.825231            0.782407    0.809091   0.753773
sweepy-sweep-203_epoch-36        0.696706  0.748091     0.714919           0.87556        0.824296  0.924769            0.709201    0.879545   0.702465
# sweepy-sweep-203_epoch-37        0.688831  0.741755     0.715146           0.882758       0.809884  0.936632            0.744792    0.873377   0.703597
# visionary-sweep-121_epoch-145    0.761269  0.711066     0.707736           0.811482       0.861477  0.721354            0.755787    0.817208   0.69668
visionary-sweep-121_epoch-148    0.761513  0.711151     0.706827           0.81222        0.862892  0.720486            0.755498    0.817695   0.69756
warm-sweep-160_epoch-10          0.713731  0.710125     0.725748           0.863731       0.831929  0.86169             0.743924    0.706494   0.720951
warm-sweep-160_epoch-11          0.705324  0.707333     0.725916           0.867242       0.823572  0.883681            0.739583    0.713799   0.723089
comic-sweep-29_epoch-20          0.725643  0.751254     0.723035           0.816265       0.808009  0.911748            0.720197    0.818182   0.756791
comic-sweep-29_epoch-24          0.745209  0.754502     0.718756           0.837658       0.8189    0.90191             0.725984    0.816315   0.783702


In [357]:
comic-sweep-29_epoch-24         comic      deep        comic                sweepy        visionary    sweepy           fearless    sweepy     fearless

In [135]:
set({'biophony':'visionary-sweep-121_epoch-145',
'bird':'deep-sweep-198_epoch-14',
'songbirds':'comic-sweep-29_epoch-20 ',
'duck-goose-swan':'sweepy-sweep-203_epoch-37',
'anthrophony':'visionary-sweep-121_epoch-145',
'insect':'sweepy-sweep-203_epoch-37',
'grouse-ptarmigan':'fearless-sweep-206_epoch-66',
'aircraft':'sweepy-sweep-203_epoch-37',
'silence':'fearless-sweep-206_epoch-66'}.values())



{'comic-sweep-29_epoch-20 ',
 'deep-sweep-198_epoch-14',
 'fearless-sweep-206_epoch-66',
 'sweepy-sweep-203_epoch-37',
 'visionary-sweep-121_epoch-145'}

In [ ]:
(valid) model_name      biophony    bird    songbirds    duck-goose-swan    anthrophony    insect    grouse-ptarmigan    aircraft    silence
--------------------  ----------  ------  -----------  -----------------  -------------  --------  ------------------  ----------  ---------
none                        0.75    0.79         0.72               0.88           0.86       0.9                0.78        0.87       0.78



In [ ]:
'deep-sweep-198/best_model_14_min_ROC_AUC=0.6982.pt',
elif 'deep-sweep-198' in model_name:
    config['CNN_filters_1'] = 5
    config['CNN_kernel_size'] = 12
    config['fc_1_size'] = 84

'fearless-sweep-206/best_model_64_min_ROC_AUC=0.6955.pt',

elif 'fearless-sweep-206' in model_name:
    config['CNN_filters_1'] = 5
    config['CNN_kernel_size'] = 12
    config['fc_1_size'] = 115
        
'sweepy-sweep-203/best_model_37_min_ROC_AUC=0.7036.pt'

elif 'sweepy-sweep-203' in model_name:
    config['CNN_filters_1'] = 12
    config['CNN_kernel_size'] = 12
    config['fc_1_size'] = 87
        
'visionary-sweep-121/best_model_145_min_ROC_AUC=0.6967.pt',

elif 'visionary-sweep-121' in model_name:
    config['CNN_filters_1'] = 4
    config['CNN_kernel_size'] = 12
    config['fc_1_size'] = 84

In [359]:
multi_label_vector[0],set_names[0]

([1, 0, 0, 0, 0, 0, 0, 0, 0], 'valid')

In [361]:
target_taxo_names_header,target_taxo_header

('biophony_pred,bird_pred,songbirds_pred,duck-goose-swan_pred,anthrophony_pred,insect_pred,grouse-ptarmigan_pred,aircraft_pred,silence_pred',
 'biophony,bird,songbirds,duck-goose-swan,anthrophony,insect,grouse-ptarmigan,aircraft,silence')

In [362]:
# audio_index,file_paths
# with open('/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_comic-sweep-29_activations-fc1_labels_v2.csv','w') as f:
#     f.writelines('sample_id,file_path,part_index,set_name,location_id,timestamp,'+(','.join(target_taxo))+'\n')
#     for i,label_vector in enumerate(multi_label_vector):
#         label_vector = [str(i) for i in label_vector]
#         t = timestamps[i].strftime('%Y-%m-%d_%H:%M:%S')
#         f.writelines(f'{i},{file_paths[i]},{audio_index[i]},{t},{location_id_info[i]},{set_names[i]},'+(','.join(label_vector))+'\n')

In [72]:
# outputs_sigmoid_full_list[0]

In [488]:
# audio_index,file_paths
# pathmap.out_csv_file_name
with open('test20.csv','w') as f:
    f.writelines('sample_id,file_path,part_index,timestamp,location_id,set_name,'+target_taxo_header+','+target_taxo_names_header+'\n')
    for i,label_vector in enumerate(multi_label_vector):
        label_vector = [str(i) for i in label_vector]
        label_vector=','.join(label_vector)
        pred_vector = outputs_full_list[i]
        t = timestamps[i].strftime('%Y-%m-%d_%H:%M:%S')
        f.writelines(f'{i},{file_paths[i]},{audio_index[i]},{t},{location_id_info[i]},{set_names[i]},{(label_vector)},'+(','.join(pred_vector))+'\n')

In [507]:
# audio_index,file_paths
# pathmap.out_csv_file_name
with open('testsigmoidfull.csv','w') as f:
    f.writelines('sample_id,file_path,part_index,timestamp,location_id,set_name,'+target_taxo_header+','+target_taxo_names_header+'\n')
    for i,label_vector in enumerate(multi_label_vector):
        label_vector = [str(i) for i in label_vector]
        label_vector=','.join(label_vector)
        pred_vector = outputs_sigmoid_full_list[i]
        t = timestamps[i].strftime('%Y-%m-%d_%H:%M:%S')
        f.writelines(f'{i},{file_paths[i]},{audio_index[i]},{t},{location_id_info[i]},{set_names[i]},{(label_vector)},'+(','.join(pred_vector))+'\n')

In [365]:
import pandas as pd
import numpy as np

def roc_auc_perClass_compute_fn(y_pred, y_true):
    try:
        from sklearn.metrics import roc_auc_score
    except ImportError:
        raise RuntimeError(
            "This contrib module requires sklearn to be installed.")

    res = roc_auc_score(y_true, y_pred, average=None)
    return res

In [322]:
actual

['biophony',
 'bird',
 'songbirds',
 'duck-goose-swan',
 'anthrophony',
 'insect',
 'grouse-ptarmigan',
 'aircraft',
 'silence']

In [509]:
# valid[actual]

In [510]:
# valid[preds]

In [393]:
preds_np=np.array(valid[preds])
y_true_np=np.array(valid[actual])  

In [437]:
torch.sigmoid(torch.tensor(np.array(valid[preds])))


tensor([[6.6819e-01, 3.7327e-02, 2.4049e-01,  ..., 2.5477e-03, 2.3661e-02,
         1.7612e-03],
        [9.4931e-01, 6.1203e-03, 8.9244e-03,  ..., 6.3579e-04, 1.3674e-05,
         4.6176e-04],
        [2.4601e-01, 2.5831e-02, 4.1824e-01,  ..., 2.2603e-03, 6.7597e-03,
         1.0478e-03],
        ...,
        [9.9844e-01, 9.4532e-01, 2.4787e-01,  ..., 1.2026e-01, 4.9933e-07,
         5.1074e-08],
        [9.9811e-01, 9.7516e-01, 9.5391e-01,  ..., 3.1269e-04, 2.7262e-05,
         1.1367e-07],
        [9.9508e-01, 9.9081e-01, 4.5017e-01,  ..., 3.8573e-04, 1.6621e-07,
         2.9829e-05]], dtype=torch.float64)

In [ ]:
Biophony: 0.6342
Bird: 0.7459
Songbirds: 0.7204
Duck.goose.swan: 0.773
Anthrophony: 0.5855
Insect: 0.9115
Grouse.Ptarmigan: 0.7771
Aircraft: 0.526
Silence: 0.4974

In [ ]:
valid_comic-sweep-29_epoch-20        0.725643  0.751254  0.723035  0.816265  0.808009  0.911748  0.720197  0.818182  0.756791
test_comic-sweep-29_epoch-20         0.692176  0.6844    0.35647   0.686799  0.801752  0.624946  0.620627  0.587307  0.694296
valid_comic-sweep-29_epoch-24        0.745209  0.754502  0.718756  0.837658  0.8189    0.90191   0.725984  0.816315  0.783702
test_comic-sweep-29_epoch-24         0.652705  0.617572  0.366295  0.708022  0.729618  0.598996  0.673596  0.516208  0.674526

In [132]:
with open('test_pre.csv','w') as f:
    f.writelines('sample_id,file_path,part_index,timestamp,location_id,set_name,'+target_taxo_header+','+target_taxo_names_header+'\n')
    for i,label_vector in enumerate(multi_label_vector):
        label_vector = [str(i) for i in label_vector]
        label_vector=','.join(label_vector)
        pred_vector = [f'{float(i):.6f}' for i in outputs_sigmoid_full_list[i]]
        t = timestamps[i].strftime('%Y-%m-%d_%H:%M:%S')
        f.writelines(f'{i},{file_paths[i]},{audio_index[i]},{t},{location_id_info[i]},{set_names[i]},{(label_vector)},'+(','.join(pred_vector))+'\n')

    
asd=pd.read_csv('test_pre.csv')
preds=[i for i in asd.columns if 'pred' in i]
actual=[i.split('_')[0] for i in preds ]
valid=asd[asd['set_name']=='valid']
kk=roc_auc_perClass_compute_fn(np.array(valid[preds]),np.array(valid[actual]))
print(pathmap.out_csv_file_name)
print(kk)

/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_comic-sweep-29_best_model_24_min_ROC_AUC=0.7188.pt_labels_v0.csv
[0.74554126 0.75450228 0.71874402 0.83781034 0.81888326 0.90190972
 0.73133681 0.81635552 0.78420523]


In [125]:
outputs_sigmoid_full_list[0]

['0.6690313816070557',
 '0.037341345101594925',
 '0.2412663996219635',
 '0.017971660941839218',
 '0.8155879378318787',
 '0.47860562801361084',
 '0.002548219170421362',
 '0.02363586612045765',
 '0.0017552142962813377']

In [151]:
# for i,label_vector in enumerate(multi_label_vector):
#     for i in outputs_sigmoid_full_list[i]:
#         if 'e' in i:
#             print(i)
# #     pred_vector = [i[:] for i in outputs_sigmoid_full_list[i]]

In [485]:
shared='/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_comic-sweep-29_activations-fc1_labels_v4.csv'
asd=pd.read_csv(shared)
preds=[i for i in asd.columns if 'pred' in i]
actual=[i.split('_')[0] for i in preds ]
valid=asd[asd['set_name']=='valid']
kk=roc_auc_perClass_compute_fn(np.array(valid[preds]),np.array(valid[actual]))
# print(pathmap.out_csv_file_name)
print(kk)

[0.63421695 0.74593465 0.72039348 0.77295737 0.58545012 0.91145833
 0.7770544  0.52597403 0.49735915]


In [486]:

shared='/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_comic-sweep-29_activations-fc1_labels_v4.csv'
asd=pd.read_csv(shared)
preds=[i for i in asd.columns if 'pred' in i]
actual=[i.split('_')[0] for i in preds ]
valid=asd[asd['set_name']=='valid']
kk=roc_auc_perClass_compute_fn(torch.sigmoid(torch.tensor(np.array(valid[preds]))),np.array(valid[actual]))
# print(pathmap.out_csv_file_name)
print(kk)

[0.63421695 0.74593465 0.72039348 0.77295737 0.58545012 0.91145833
 0.7770544  0.52597403 0.49735915]


In [508]:
asd=pd.read_csv('testsigmoidfull.csv')
preds=[i for i in asd.columns if 'pred' in i]
actual=[i.split('_')[0] for i in preds ]
valid=asd[asd['set_name']=='valid']
kk=roc_auc_perClass_compute_fn(np.array(valid[preds]),np.array(valid[actual]))
print(pathmap.out_csv_file_name)
print(kk)

/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_comic-sweep-29_best_model_20_min_ROC_AUC=0.7202.pt_labels_v0.csv
[0.7256433  0.7512538  0.723035   0.81626475 0.80800869 0.91174769
 0.72019676 0.81818182 0.75679074]


In [458]:
asd=pd.read_csv('test.csv')
preds=[i for i in asd.columns if 'pred' in i]
actual=[i.split('_')[0] for i in preds ]
valid=asd[asd['set_name']=='valid']
kk=roc_auc_perClass_compute_fn(torch.sigmoid(torch.tensor(np.array(valid[preds]))),np.array(valid[actual]))
print(pathmap.out_csv_file_name)
print(kk)

/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_comic-sweep-29_best_model_24_min_ROC_AUC=0.7188.pt_labels_v0.csv
[0.74524179 0.75451178 0.71875598 0.83760684 0.81886681 0.90190972
 0.72583912 0.81627435 0.78351358]


In [443]:
asd=pd.read_csv('test.csv')
preds=[i for i in asd.columns if 'pred' in i]
actual=[i.split('_')[0] for i in preds ]
valid=asd[asd['set_name']=='valid']
kk=roc_auc_perClass_compute_fn(sigmoid(np.array(valid[preds])),np.array(valid[actual]))
print(pathmap.out_csv_file_name)
print(kk)

/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_comic-sweep-29_best_model_24_min_ROC_AUC=0.7188.pt_labels_v0.csv
[0.74524179 0.75451178 0.71875598 0.83760684 0.81886681 0.90190972
 0.72583912 0.81627435 0.78351358]


In [448]:
f"{sigmoid(np.array(valid[preds]))[1][0]:.2f}"

'0.95'

In [449]:
sigmoid(np.array(valid[preds]))[1][0]

0.9493096750687493

In [107]:
# /scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_warm-sweep-160_best_model_11_min_ROC_AUC=0.7076.pt_labels_v0.csv
# [0.63068988 0.68847834 0.70840505 0.83750509 0.81001579 0.83695023
#  0.74117477 0.4875     0.49823944]

# /scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_warm-sweep-160_best_model_10_min_ROC_AUC=0.7065.pt_labels_v0.csv
# [0.656189   0.69231573 0.7028232  0.84581044 0.82189392 0.75245949
#  0.81163194 0.49375    0.49911972]


# /scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_visionary-sweep-121_best_model_148_min_ROC_AUC=0.6976.pt_labels_v0.csv
# [0.56852263 0.66474164 0.69940476 0.68620269 0.51625428 0.5240162
#  0.47569444 0.5        0.49647887]

# /scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_visionary-sweep-121_best_model_145_min_ROC_AUC=0.6967.pt_labels_v0.csv
# [0.57481145 0.66358283 0.68593421 0.68615181 0.51533298 0.52546296
#  0.47569444 0.5        0.49647887]


# /scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_sweepy-sweep-203_best_model_37_min_ROC_AUC=0.7036.pt_labels_v0.csv
# [0.50706522 0.67740312 0.70496271 0.7847095  0.4971374  0.88845486
#  0.49045139 0.5        0.49735915]


# /scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_sweepy-sweep-203_best_model_36_min_ROC_AUC=0.7025.pt_labels_v0.csv
# [0.50174135 0.69028305 0.70960031 0.78414988 0.50478744 0.87731481
#  0.49045139 0.5        0.49735915]


# /scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_fearless-sweep-206_best_model_66_min_ROC_AUC=0.6975.pt_labels_v0.csv
# [0.49081633 0.6875     0.64633534 0.69288004 0.58207752 0.63686343
#  0.80801505 0.49285714 0.49823944]

# /scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_deep-sweep-198_best_model_14_min_ROC_AUC=0.6982.pt_labels_v0.csv
# [0.71947649 0.78992211 0.7176922  0.69301994 0.78285404 0.83969907
#  0.7572338  0.65105519 0.59400151]

# /scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_deep-sweep-198_best_model_12_min_ROC_AUC=0.7000.pt_labels_v0.csv
# [0.71315439 0.79289514 0.72886785 0.69170991 0.73715122 0.79427083
#  0.75911458 0.63348214 0.59557344]


# /scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_dandy-sweep-196_best_model_67_min_ROC_AUC=0.7062.pt_labels_v0.csv
# [0.48979592 0.69020707 0.68544416 0.67560796 0.66800474 0.80063657
#  0.62934028 0.49732143 0.49823944]

# /scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_dandy-sweep-196_best_model_68_min_ROC_AUC=0.7063.pt_labels_v0.csv
# [0.51574978 0.69265767 0.67760327 0.67920737 0.68733548 0.79239005
#  0.64351852 0.49196429 0.49735915]


In [ ]:
[0.71315439 0.79289514 0.72886785 0.84581044 0.78285404 0.88845486 
0.81163194 0.65105519  0.49735915]
['biophony',
 'bird', ok
 'songbirds',
 'duck-goose-swan', ok
 'anthrophony', ook
 'insect', ok 
 'grouse-ptarmigan', ok
 'aircraft',
 'silence']

In [108]:
# from nna.exp.megan import preparedataset
# preparedataset.load_taxonomy2dataset(('/home/enis/projects/nna/src/nna/assets/taxonomy/taxonomy.yaml'), audio_dataset)


'biophony'

In [115]:
# audio_dataset.update_samples_w_clipping_info(
#     output_folder=pathmap.clipping_results_path)
# # print('inference part')
# for audio_ins in audio_dataset.values():
# #     if is_result_exist(audio_ins, label_names, v_str,
# #                        file_properties_df, pathmap):
# #         continue
#     dataloader = prepare_dataloader_from_audio_ins(
#         audio_ins, config, transformCompose)
#     outputs = single_file_inference(dataloader, config, model_saved)
#     break
    
# #     save_results_disk(outputs, audio_ins, label_names, v_str,
# #                       file_properties_df, pathmap)

In [ ]:
region_location_datasets = load_audio_files(region_location,
                                            file_properties_df)
label_names = config['label_names']
v_str = config['v_str']
# for region, location in region_location:
#     print(region, location)
#     region_location_ins = region_location_datasets[(region, location)]
    region_location_ins.update_samples_w_clipping_info(
        output_folder=pathmap.clipping_results_path)
    # print('inference part')
    for audio_ins in region_location_ins.values():
        if is_result_exist(audio_ins, label_names, v_str,
                           file_properties_df, pathmap):
            continue
        dataloader = prepare_dataloader_from_audio_ins(
            audio_ins, config, transformCompose)
        outputs = single_file_inference(dataloader, config, model_saved)
        save_results_disk(outputs, audio_ins, label_names, v_str,
                          file_properties_df, pathmap)

In [ ]:
# another thing
# with open('/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite/files_filtered_v3_info.csv','w') as f:
#     f.writelines(f'file_name,taxo_code,location_id\n')
#     for file_name in cached_dict.keys():
# #         print(file_name)
#         audio=audio_dataset[file_name.split('/')[-1]]
#         taxo_code=audio.taxo_code
#         location_id=audio.location_id
        
#         f.writelines(f'{file_name},{taxo_code},{location_id}\n')

In [370]:
from sklearn.metrics import roc_auc_score

In [422]:
y_true=[1,1,1,1,0,0,0,0,1,1]
y_pred=[1,-1,-221,1,2,3,-4,-2,1,1]
y_pred_s=sigmoid(np.array(y_pred))

ro=roc_auc_score(y_true, y_pred, average=None)
ro_s = roc_auc_score(y_true, y_pred_s, average=None)
print(ro,ro_s)

0.4166666666666667 0.4166666666666667


In [419]:
preds_np,y_true_np

(array([[  0.7 ,  -3.25,  -1.15, ...,  -5.97,  -3.72,  -6.34],
        [  2.93,  -5.09,  -4.71, ...,  -7.36, -11.2 ,  -7.68],
        [ -1.12,  -3.63,  -0.33, ...,  -6.09,  -4.99,  -6.86],
        ...,
        [  6.46,   2.85,  -1.11, ...,  -1.99, -14.51, -16.79],
        [  6.27,   3.67,   3.03, ...,  -8.07, -10.51, -15.99],
        [  5.31,   4.68,  -0.2 , ...,  -7.86, -15.61, -10.42]]),
 array([[1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]))

In [404]:
roc_auc_score(y_true_np[:,0], preds_np[:,0], average=None)

0.7452417923691216

In [401]:
y_true_np[:,0].shape

(582,)

In [412]:
preds_np[:,0][:10]

array([ 0.7 ,  2.93, -1.12,  1.54, -0.03, -0.95,  1.61,  4.92,  5.97,
        5.9 ])

In [413]:
sigmoid(preds_np[:,0])[:10]

array([0.66818777, 0.94930968, 0.24601128, 0.82346473, 0.49250056,
       0.27888482, 0.83341139, 0.99275376, 0.99745227, 0.99726804])

In [426]:
roc_auc_score(y_true, sigmoid(preds_np[:,1])[:10], average=None),roc_auc_score(y_true, (preds_np[:,1])[:10], average=None)


(0.2916666666666667, 0.2916666666666667)

In [418]:
sigmoid()

AttributeError: 'list' object has no attribute 'shape'

TypeError: sigmoid() missing 1 required positional arguments: "input"

In [175]:
asd='/scratch/enis/data/nna/database/allFields_dataV5.pkl'
bcd=pd.read_pickle(asd)

In [178]:
kk=bcd[bcd['year']=='2020']
kk[kk['site_id']=='14']

,site_id,locationId,site_name,recorderId,hour_min_sec,year,month,day,region,timestamp,durationSec,timestampEnd
/tank/data/nna/real/dempster/14/2020/S4A10424_20200812_160000.flac,14,14,,S4A10424,160000,2020,08,12,dempster,2020-08-12 16:00:00,300.0,2020-08-12 16:05:00
/tank/data/nna/real/dempster/14/2020/S4A10424_20200916_052000.flac,14,14,,S4A10424,052000,2020,09,16,dempster,2020-09-16 05:20:00,300.0,2020-09-16 05:25:00
/tank/data/nna/real/dempster/14/2020/S4A10424_20200427_201234.flac,14,14,,S4A10424,201234,2020,04,27,dempster,2020-04-27 20:12:34,300.0,2020-04-27 20:17:34
/tank/data/nna/real/dempster/14/2020/S4A10424_20200916_184000.flac,14,14,,S4A10424,184000,2020,09,16,dempster,2020-09-16 18:40:00,300.0,2020-09-16 18:45:00
/tank/data/nna/real/dempster/14/2020/S4A10424_20200825_104000.flac,14,14,,S4A10424,104000,2020,08,25,dempster,2020-08-25 10:40:00,300.0,2020-08-25 10:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...
/tank/data/nna/real/dempster/14/2020/S4A10424_20200429_025234.flac,14,14,,S4A10424,025234,2020,04,29,dempster,2020-04-29 02:52:34,300.0,2020-04-29 02:57:34
/tank/data/nna/real/dempster/14/2020/S4A10424_20200402_025234.flac,14,14,,S4A10424,025234,2020,04,02,dempster,2020-04-02 02:52:34,300.0,2020-04-02 02:57:34
/tank/data/nna/real/dempster/14/2020/S4A10424_20200525_145234.flac,14,14,,S4A10424,145234,2020,05,25,dempster,2020-05-25 14:52:34,300.0,2020-05-25 14:57:34
/tank/data/nna/real/dempster/14/2020/S4A10424_20200523_121234.flac,14,14,,S4A10424,121234,2020,05,23,dempster,2020-05-23 12:12:34,300.0,2020-05-23 12:17:34
